# **HW2 - Text Classification**

## 1. IMDB Dataset

In [48]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [49]:
from keras.datasets import imdb
INDEX_FROM=3
(x_train, y_train), (x_test, y_test) = imdb.load_data(index_from=INDEX_FROM)

In [50]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 2. Pre-Processing

### 2.1. Any data cleaning

In [51]:
import string
train_sents=[]
test_sents=[]
word_to_id = imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
word_to_id["<UNUSED>"] = 3
id_to_word = {value:key for key,value in word_to_id.items()}
for j in range(0,len(x_train)):
  decoded =  [ id_to_word[id] for id in x_train[j] if id_to_word[id] != "br" and  not id_to_word[id] in stop_words and not (id_to_word[id].isdigit()) and not id_to_word[id] in string.punctuation] 
  train_sents.append(decoded)

for j in range(0,len(x_test)):
  decoded = [id_to_word[id] for id in x_test[j] if id_to_word[id] != "br" and  not id_to_word[id] in stop_words  and not (id_to_word[id].isdigit()) and not id_to_word[id] in string.punctuation ] 
  test_sents.append(decoded)

### 2.2. Tokenization

### 2.3. Stemming

In [52]:
import nltk
from nltk.stem.snowball import SnowballStemmer
snow_stemmer = SnowballStemmer(language='english')
train_sttemed_sents=[]
test_sttemed_sents=[]
for sent in train_sents:
    tmp = [snow_stemmer.stem(w) for w in sent]
    train_sttemed_sents.append(tmp)

for sent in test_sents:
    tmp = [snow_stemmer.stem(w) for w in sent]
    test_sttemed_sents.append(tmp)

## 3. Build Models

### 3.1. Uni-Gram

In [53]:
count_neg = 0
count_pos = 0
for i, tmp in enumerate(train_sttemed_sents):
      if y_train[i]==0:
             count_pos += len(tmp)
      elif y_train[i]==1 :
             count_neg += len(tmp)
unigram = {'neg':{}, 'pos':{}}
unigram_prop = {'neg':{}, 'pos':{}}

for i, sample in enumerate(train_sttemed_sents):
    if y_train[i] == 0:
        key = 'neg'
        n_key = 'pos'
    elif y_train[i] == 1 :
        key = 'pos'
        n_key = 'neg'

    fdist_item = nltk.FreqDist(sample).items()
    for k, v in fdist_item :
        if k in unigram[key].keys():
            unigram[key][k] += v
        else:
            unigram[key][k] = v

        if not k in unigram[n_key].keys():
            unigram[n_key][k] = 0

unigram_prop['neg'] = {k : (v + 1) / (count_neg + len(unigram['neg']) + 1) for k, v in unigram['neg'].items()}
unigram_prop['neg'][-1] = 1 / (count_neg + len(unigram['neg']) + 1)
unigram_prop['pos'] = {k : (v + 1) / (count_pos + len(unigram['pos']) + 1) for k, v in unigram['pos'].items()}
unigram_prop['pos'][-1] = 1 / (count_pos + len(unigram['pos']) + 1)

### 3.2. Bi-Gram


In [54]:
# # NLTK function to generate ngrams
# import nltk
# from nltk.util import ngrams
# bi_GRAMS=[]
# for sent in train_sttemed_sents:
#   temp=ngrams(sequence= sent, n=2)
#   bi_GRAMS.append(temp)

# # for t in bi_GRAMS:
# #   for grams in t:
# #     print(grams)

bigram = {'neg':{}, 'pos':{}}
bigram_prop = {'neg':{}, 'pos':{}}
for i, sample in enumerate(train_sttemed_sents):
    if y_train[i] == 0:
        key = 'neg'
        n_key = 'pos'
    elif y_train[i] == 1 :
        key = 'pos'
        n_key = 'neg'
    bgs = nltk.bigrams(sample)
    fdist_items = nltk.FreqDist(bgs).items()
    for k, v in fdist_items:
        if k in bigram[key].keys():
            bigram[key][k] += v
        else:
            bigram[key][k] = v
        if not k in bigram[n_key].keys():
            bigram[n_key][k] = 0

bigram_prop['pos'] = {k : (v + 1) / (unigram['pos'][k[0]] + len(bigram['pos']) + 1) for k, v in bigram['pos'].items()}
bigram_prop['pos'][-1] = 1 / (unigram['pos'][k[0]] + len(bigram['pos']) + 1)
bigram_prop['neg'] = {k : (v + 1) / (unigram['neg'][k[0]] + len(bigram['neg']) + 1) for k, v in bigram['neg'].items()}
bigram_prop['neg'][-1] = 1 / (unigram['neg'][k[0]] + len(bigram['neg']) + 1)

### 3.3. Tri-Gram

In [55]:
# # NLTK function to generate ngrams
# import nltk
# from nltk.util import ngrams
# tri_GRAMS=[]
# for sent in train_sttemed_sents:
#   temp=ngrams(sequence= sent, n=3)
#   tri_GRAMS.append(temp)
trigram = {'neg':{}, 'pos':{}}
trigram_prop = {'neg':{}, 'pos':{}}
for i, sample in enumerate(train_sttemed_sents):
    if y_train[i] == 0:
        key = 'neg'
        n_key = 'pos'
    elif y_train[i] == 1 :
        key = 'pos'
        n_key = 'neg'
    bgs = nltk.trigrams(sample)
    fdist_items = nltk.FreqDist(bgs).items()
    for k, v in fdist_items:
        if k in trigram[key].keys():
            trigram[key][k] += v
        else:
            trigram[key][k] = v
        if not k in trigram[n_key].keys():
            trigram[n_key][k] = 0
trigram_prop['pos'] = {k : (v + 1) / (bigram['pos'][(k[0], k[1])] + len(trigram['pos']) + 1) for k, v in trigram['pos'].items()}
trigram_prop['pos'][-1] = 1 / (bigram['pos'][(k[0], k[1])] + len(trigram['pos']) + 1)
trigram_prop['neg'] = {k : (v + 1) / (bigram['neg'][(k[0], k[1])] + len(trigram['neg']) + 1) for k, v in trigram['neg'].items()}
trigram_prop['neg'][-1] = 1 / (bigram['neg'][(k[0], k[1])] + len(trigram['neg']) + 1)

## 4.  Naive Beyes

In [56]:
from nltk import bigrams,trigrams,FreqDist
def NB(ngram_type,probs,arr):
    neg_pred = len(y_train[y_train==0]) / len(y_train)
    pos_pred = len(y_train[y_train==1]) / len(y_train)

    if ngram_type == 1:
        ngram = arr
    elif ngram_type == 2:
        ngram = bigrams(arr)
    elif  ngram_type == 3 :
        ngram = trigrams(arr)
    else:
      return
    fdist = FreqDist(ngram)

    for k, v in fdist.items():
        if k in probs['neg']:
            neg_pred *= probs['neg'][k]
        else:
            neg_pred *= probs['neg'][-1]
        if k in probs['pos']:
            pos_pred *= probs['pos'][k]
        else:
            pos_pred *= probs['pos'][-1]
        if pos_pred < 10**(-10) or neg_pred < 10**(-10):
            pos_pred *= 10**(10)
            neg_pred *= 10**(10)
        elif pos_pred > 10**10 or neg_pred > 10**10:
            pos_pred *= 10**(-10)
            neg_pred *= 10**(-10)
       

    return neg_pred, pos_pred



## 5. Evaluate Model

In [57]:
#unigram
preds_uni = []
for sample in test_sttemed_sents:
        neg_pred, pos_pred = NB(1, unigram_prop, sample)
        if neg_pred > pos_pred:
            preds_uni.append(0)
        else:
            preds_uni.append(1)
y_pred_unigram=np.array(preds_uni)

#bigram
preds_bi = []
for sample in test_sttemed_sents:
        neg_pred, pos_pred = NB(2, bigram_prop, sample)
        if neg_pred > pos_pred:
            preds_bi.append(0)
        else:
            preds_bi.append(1)
y_pred_bigram=np.array(preds_bi)
#trigram
preds_tri = []
for sample in test_sttemed_sents:
        neg_pred, pos_pred = NB(3, trigram_prop, sample)
        if neg_pred > pos_pred:
            preds_tri.append(0)
        else:
            preds_tri.append(1)
y_pred_trigram=np.array(preds_tri)

In [58]:
print('Precision For unigram:', "{:.3f}".format(precision_score(y_test, y_pred_unigram)))
print('Precision For bigram:', "{:.3f}".format(precision_score(y_test, y_pred_bigram)))
print('Precision For trigram', "{:.3f}".format(precision_score(y_test, y_pred_trigram)))
print("//////////////////////////////")
print('Recall For unigram:', "{:.3f}".format(recall_score(y_test, y_pred_unigram)))
print('Recall For bigram:',"{:.3f}".format(recall_score(y_test, y_pred_bigram)))
print('Recall For trigram', "{:.3f}".format(recall_score(y_test, y_pred_trigram)))
print("//////////////////////////////")
print('F1 Score For unigram:', "{:.3f}".format(f1_score(y_test, y_pred_unigram)))
print('F1 Score For bigram:',"{:.3f}".format(f1_score(y_test, y_pred_bigram)))
print('F1 Score For trigram', "{:.3f}".format(f1_score(y_test, y_pred_trigram)))
print("//////////////////////////////")
print('Accuracy For unigram:', "{:.3f}".format(accuracy_score(y_test, y_pred_unigram)))
print('Accuracy For bigram:',"{:.3f}".format(accuracy_score(y_test, y_pred_bigram)))
print('Accuracy For trigram', "{:.3f}".format(accuracy_score(y_test, y_pred_trigram)))

Precision For unigram: 0.753
Precision For bigram: 0.873
Precision For trigram 0.792
//////////////////////////////
Recall For unigram: 0.922
Recall For bigram: 0.831
Recall For trigram 0.721
//////////////////////////////
F1 Score For unigram: 0.829
F1 Score For bigram: 0.852
F1 Score For trigram 0.755
//////////////////////////////
Accuracy For unigram: 0.810
Accuracy For bigram: 0.855
Accuracy For trigram 0.766


In [59]:
for idx, sample in enumerate(test_sttemed_sents):
    neg_pred, pos_pred = NB(1, unigram_prop, test_sttemed_sents[idx])
    if (y_test[idx] == 1)!= (neg_pred < pos_pred):
      print(idx)
      print(test_sttemed_sents[idx])
      print('y_test:', y_test[idx] == 1)
      neg_pred, pos_pred = NB(1, unigram_prop, test_sttemed_sents[idx])
      print('unigram:', "{:.20f}".format(neg_pred), "{:.20f}".format(pos_pred), neg_pred < pos_pred)
      neg_pred, pos_pred = NB(2, bigram_prop, test_sttemed_sents[idx])
      print('bigram:', "{:.20f}".format(neg_pred), "{:.20f}".format(pos_pred), neg_pred < pos_pred)
      neg_pred, pos_pred = NB(3, trigram_prop, test_sttemed_sents[idx])
      print('trigram:', "{:.20f}".format(neg_pred), "{:.20f}".format(pos_pred), neg_pred < pos_pred)
      print('####################')
      break

for idx, sample in enumerate(test_sttemed_sents):
    neg_pred, pos_pred = NB(2, bigram_prop,test_sttemed_sents[idx])
    if bool(y_test[idx] == 1)!= bool(neg_pred < pos_pred) :
      print(idx)
      print(test_sttemed_sents[idx])
      print('y_test:', y_test[idx] == 1)
      neg_pred, pos_pred = NB(1, unigram_prop, test_sttemed_sents[idx])
      print('unigram:', "{:.20f}".format(neg_pred), "{:.20f}".format(pos_pred), neg_pred < pos_pred)
      neg_pred, pos_pred = NB(2, bigram_prop,test_sttemed_sents[idx])
      print('bigram:', "{:.20f}".format(neg_pred), "{:.20f}".format(pos_pred), neg_pred < pos_pred)
      neg_pred, pos_pred = NB(3, trigram_prop, test_sttemed_sents[idx])
      print('trigram:', "{:.20f}".format(neg_pred), "{:.20f}".format(pos_pred), neg_pred < pos_pred)
      print('####################')
      break

for idx, sample in enumerate(test_sttemed_sents):
    neg_pred, pos_pred = NB(3, trigram_prop,test_sttemed_sents[idx])
    neg_pred1, pos_pred1 = NB(1,unigram_prop,test_sttemed_sents[idx])
    if bool(neg_pred1 < pos_pred1) == bool(y_test[idx] == 1) and bool(neg_pred < pos_pred) != bool(y_test[idx] == 1)  :
      print(idx)
      print(test_sttemed_sents[idx])
      print('y_test:', y_test[idx] == 1)
      neg_pred, pos_pred = NB(1, unigram_prop, test_sttemed_sents[idx])
      print('unigram:', "{:.20f}".format(neg_pred), "{:.20f}".format(pos_pred), neg_pred < pos_pred)
      neg_pred, pos_pred = NB(2, bigram_prop,test_sttemed_sents[idx])
      print('bigram:', "{:.20f}".format(neg_pred), "{:.20f}".format(pos_pred), neg_pred < pos_pred)
      neg_pred, pos_pred = NB(3, trigram_prop,test_sttemed_sents[idx])
      print('trigram:', "{:.20f}".format(neg_pred), "{:.20f}".format(pos_pred), neg_pred < pos_pred)
      print('####################')
      break

7
['<start>', 'emperor', 'richard', 'haydn', 'dog', 'betroth', 'johanna', 'joan', 'fontain', 'dog', 'howev', 'virgil', 'bing', 'crosbi', 'arriv', 'town', 'sell', 'record', 'player', 'emperor', 'dog', 'attack', 'johanna', 'dog', 'reveng', 'attack', 'virgil', 'banish', 'town', 'psychoanalyst', 'insist', 'johanna', 'dog', 'must', 'confront', 'dog', 'overcom', 'doggi', 'fear', 'arrang', 'dog', 'fall', 'love', 'virgil', 'johanna', 'rest', 'film', 'pass', 'romanc', 'end', 'johanna', 'dog', 'give', 'birth', 'father', 'dog', 'stori', 'weak', 'vehicl', 'use', 'tri', 'creat', 'stori', 'human', 'terribl', 'storylin', 'main', 'music', 'piec', 'rubbish', 'bad', 'song', 'dread', 'choreographi', 'extrem', 'bore', 'film', 'bing', 'mani', 'word', 'sentenc', 'deliv', 'almost', 'irrit', 'manner', 'funni', 'ever', 'meant', 'bing', 'joan', 'done', 'much', 'better']
y_test: False
unigram: 0.00000000018930282923 0.00000002048303390717 True
bigram: 2365.02400839971369350678 0.00109233333028247847 False
trigra

## Good Luck!